# Recurrent Neural Networks
In this exercise, we will implement a simple one-layer recurrent neural network. We will use the formula for an [Elman RNN](https://en.wikipedia.org/wiki/Recurrent_neural_network#Elman_networks_and_Jordan_networks), one of the most basic and classical RNNs. The hidden state update and output at time $t$ are defined like this:

$$
\begin{align}
h_t &= \tanh(W_xh x_t + W_hh h_{t-1} + b_h) \\
y_t &= \tanh(W_hy h_t + b_y)
\end{align}
$$

In [1]:
import torch
import torch.nn as nn

We start by defining the RNN as a subclass of `nn.Module`. The network's parameters are created in the `__init__` method. Use `input_dim`, `hidden_dim` and `output_dim` as arguments that define the dimensionality of the input/hidden/output vectors. Define your parameters as `nn.Parameter` with the appropriate dimensions. The documentation of `torch.nn` can be found [here](https://pytorch.org/docs/stable/nn.html).

In [2]:
class RNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim

    self.Wxh = nn.Parameter(torch.zeros(hidden_dim, input_dim))
    self.bh = nn.Parameter(torch.zeros(hidden_dim))

    self.Whh = nn.Parameter(torch.zeros(hidden_dim, hidden_dim))

    self.Why = nn.Parameter(torch.zeros(output_dim, hidden_dim))
    self.by = nn.Parameter(torch.zeros(output_dim))

Add a function `reset_parameters` that initializes your parameters. Pick a suitable distribution from [nn.init](https://pytorch.org/docs/stable/nn.init.html).

In [3]:
def reset_parameters(self):
  for weight in self.parameters():
    nn.init.uniform_(weight, -1, 1)

RNN.reset_parameters = reset_parameters

In [4]:
model = RNN(5, 20, 10)
model.reset_parameters()

print(model.Wxh)

Parameter containing:
tensor([[-0.8794, -0.3084,  0.4783, -0.6229,  0.9015],
        [ 0.5838,  0.2603, -0.8658, -0.4322, -0.0829],
        [-0.1439, -0.1602,  0.1944, -0.3520,  0.4635],
        [-0.1936, -0.3334,  0.1235, -0.7335, -0.1270],
        [ 0.7728,  0.2489, -0.4591, -0.6025,  0.5094],
        [ 0.1719,  0.3454, -0.9973, -0.0361,  0.2113],
        [ 0.1997,  0.6724, -0.5541,  0.6782, -0.8514],
        [ 0.3021, -0.4854, -0.2051,  0.4267,  0.0952],
        [-0.3438,  0.3093,  0.1536, -0.6389, -0.4793],
        [ 0.7337,  0.0798, -0.3591, -0.3025,  0.7229],
        [ 0.6650, -0.2927, -0.7599, -0.6463,  0.2020],
        [-0.6908, -0.3915,  0.8679, -0.3728, -0.8711],
        [ 0.3564, -0.0056, -0.1510,  0.1929,  0.9858],
        [-0.3685, -0.8489, -0.7031,  0.6127, -0.6932],
        [ 0.0698, -0.8748, -0.7601, -0.3176, -0.2380],
        [-0.7792,  0.7785,  0.5267,  0.6726, -0.7876],
        [-0.5802, -0.3659, -0.9741,  0.0247, -0.2576],
        [-0.0381,  0.9435,  0.0661,  0.8237

Add a `forward` function that takes an input and a starting hidden state $h_{t-1}$ and returns the updated hidden state $h_t$ and output $y$ as outputs. The initial hidden state $h_0$ can be initialized randomly/to all zeros.

$$
\begin{align}
h_t &= \tanh(W_xh x_t + W_hh h_{t-1} + b_h) \\
y_t &= \tanh(W_hy h_t + b_y)
\end{align}
$$

In [5]:
def forward(self, x, hidden_state):
  h = torch.tanh(self.Wxh @ x + self.Whh @ hidden_state + self.bh)
  y = torch.tanh(self.Why @ h + self.by)
  return y, h

RNN.forward = forward

Test your RNN with a single input.

In [6]:
x = torch.randn(model.input_dim)
h0 = torch.zeros(model.hidden_dim)

y, h1 = model(x, h0)
print(f"y shape: {y.shape}, y: {y}")
print(f"h1 shape: {h1.shape}, h1: {h1}")

y shape: torch.Size([10]), y: tensor([ 0.8080, -0.9734, -0.4376, -0.9960, -0.3230,  0.9959, -0.4274,  0.1259,
         0.9049, -0.8011], grad_fn=<TanhBackward0>)
h1 shape: torch.Size([20]), h1: tensor([-0.9426,  0.9528,  0.1313, -0.5880,  0.5353,  0.6890, -0.1542, -0.6920,
        -0.3564,  0.5879,  0.7574, -0.8710,  0.5452, -0.7568,  0.7663, -0.8132,
         0.6316,  0.1497, -0.3587,  0.4144], grad_fn=<TanhBackward0>)


Now create an input sequence and run it through your RNN.

In [7]:
seq_length = 4
inputs = [torch.randn(model.input_dim) for _ in range(seq_length)]
hidden_state = torch.zeros(model.hidden_dim)
outputs = []

for x in inputs:
  y, new_hidden_state = model(x, hidden_state)
  hidden_state = new_hidden_state
  outputs.append(y)

print(f"Final output: {y}")
print(f"Final hidden state: {hidden_state}")

Final output: tensor([ 0.4932, -0.9999,  0.9931, -0.9750, -0.9594,  0.9912,  0.1784,  0.8249,
         0.4994, -0.7934], grad_fn=<TanhBackward0>)
Final hidden state: tensor([-0.9999,  0.6086,  0.9981, -0.8015,  0.9972,  0.9994, -0.9751,  0.9184,
        -0.9774,  0.5282,  0.9943, -0.9997,  0.9992, -0.0465,  0.9388,  0.9855,
         0.0511,  0.6732, -0.1633,  0.7621], grad_fn=<TanhBackward0>)


The final hidden state encodes all the information present in the input sequence. It can be used as a feature for classification, or to initialize a decoder RNN to do translation, for example.

Now look at PyTorch's documentation for the [`nn.RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) and the [`nn.RNNCell`](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html) classes. What is the difference between the two? What is the difference to the definition from Wikipedia we used above? Run your input sequence through both the `nn.RNN` and the `nn.RNNCell`.

Both modules implement the same underlying recurrence but at different levels of abstraction:

- `nn.RNN`:
  This module handles an entire input sequence automatically. You pass in a sequence (with shape like (seq_len, batch, input_size)), and it internally iterates over the time steps using an RNN cell.
  It also supports multi-layer networks, dropout between layers, and efficient batch processing.
- `nn.RNNCell`:
  This module implements only a single time step of the RNN. You must manually loop over your input sequence, feeding the output hidden state from one time step into the next.
  It gives you finer control if you want to customize the step-by-step processing.

In [12]:
rnn_cell = nn.RNNCell(model.input_dim, model.hidden_dim)
x = torch.randn(seq_length, model.input_dim)
hidden_state = torch.zeros(model.hidden_dim)

outputs = []
for i in range(seq_length):
  hidden_state = rnn_cell(x[i], hidden_state)
  outputs.append(hidden_state)

print(len(outputs))
print(outputs[0].shape)

4
torch.Size([20])


In [15]:
torch_rnn = nn.RNN(model.input_dim, model.hidden_dim, num_layers=1)
x = torch.randn(seq_length, model.input_dim)
hidden_state = torch.zeros(1, model.hidden_dim)

outputs, final_hidden_state = torch_rnn(x, hidden_state)

print(len(outputs))
print(outputs[0].shape)

4
torch.Size([20])
